# Tibetan to English Translation
## Setup

In [2]:
import random
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import matplotlib.pyplot as plt
import pickle

TF_GPU_ALLOCATOR = 'cuda_malloc_async'


2023-10-29 17:07:19.296169: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-29 17:07:19.296218: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-29 17:07:19.296263: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-29 17:07:19.307132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using TensorFlow backend


In [3]:
MAX_SEQUENCE_LENGTH = 45
VOCAB_SIZE = 15000

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

AUTOTUNE = tf.data.AUTOTUNE

In [4]:
with open('/home/j/Documents/Projects/Iron-Bridge/lotsawa/tokenizers/tib-tokenizer.pickle', 'rb') as handle:
    tib_tokenizer = pickle.load(handle)

with open('/home/j/Documents/Projects/Iron-Bridge/lotsawa/tokenizers/eng-tokenizer.pickle', 'rb') as handle:
    eng_tokenizer = pickle.load(handle)

2023-10-29 17:07:23.784124: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-29 17:07:23.791969: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-29 17:07:23.792432: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

### Load in Data

In [5]:
def load_data(text_file):
    with open(text_file) as f:
        lines = f.read().split("\n")[:-1]
    text_pairs = []
    for line in lines:
        try:
            tib, eng = line.split(",")[:2]
            eng = eng.lower()
            text_pairs.append((tib, eng))
        except:
            pass

    random.shuffle(text_pairs)
    num_val_samples = int(0.05 * len(text_pairs))
    num_train_samples = len(text_pairs) - num_val_samples
    train_pairs = text_pairs[:num_train_samples]
    val_pairs = text_pairs[num_train_samples :]

    print(f"{len(text_pairs)} total pairs")
    print(f"{len(train_pairs)} training pairs")
    print(f"{len(val_pairs)} validation pairs")

    return train_pairs, val_pairs

### Data Preprocessing

In [6]:
def tib_eng_preprocess_batch(tib, eng):

    tib = tib_tokenizer(tib)
    eng = eng_tokenizer(eng)
    

    # add special tokens [start] and [end] and pad tib
    tib_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length = MAX_SEQUENCE_LENGTH,
        start_value = tib_tokenizer.token_to_id("[START]"),
        end_value = tib_tokenizer.token_to_id("[END]"),
        pad_value = tib_tokenizer.token_to_id("[PAD]")
    )

    tib = tib_start_end_packer(tib)

    # pad eng to max_sequence_length
    eng_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH+1,
        pad_value = eng_tokenizer.token_to_id("[PAD]"),
    )

    eng = eng_start_end_packer(eng)



    return (
        {
        "encoder_inputs": tib,
        "decoder_inputs": eng[:, :-1]
        },
        eng[:, 1:],
    )

def make_dataset(pairs, batch_size):
    tib_texts, eng_texts = zip(*pairs)
    tib_texts = list(tib_texts)
    eng_texts = list(eng_texts)
    dataset = tf.data.Dataset.from_tensor_slices((tib_texts, eng_texts))
    dataset=dataset.batch(batch_size)
    dataset = dataset.map(tib_eng_preprocess_batch, num_parallel_calls=AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()


In [7]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length = MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(encoder_inputs)

encoder_outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim = INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)(decoder_inputs)

x = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

tib_eng_translator = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="tib_eng_translator",
)

In [13]:
tib_eng_translator = tf.keras.models.load_model("/home/j/Documents/Projects/Iron-Bridge/lotsawa/models/tib-eng-translator-0.4.1.30.keras")

tib_eng_translator.compile(
    "rmsprop", 
    loss="sparse_categorical_crossentropy", 
    metrics=["accuracy"]
)

loss_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

In [14]:
def train(tib_eng_translator):
    for i in range(31): # range beginning updated because of crash
        
        print('TRAINING BATCH ' + str(i))

        # load in batch from training batches
        text_file = '/home/j/Documents/Projects/Iron-Bridge/lotsawa/data/training-batches/training-batch-'+str(i)+'.txt'
        train_pairs, val_pairs = load_data(text_file)
        
        for j in range(2, 6):
            # make datasets from loaded data
            batch_size = 16 * (2**j)
            print('BATCH SIZE ' + str(batch_size))
            tib_eng_train_ds = make_dataset(train_pairs, batch_size)
            tib_eng_val_ds = make_dataset(val_pairs, batch_size)
            
            # fit model to dataset batch
            tib_eng_translator.fit(
            tib_eng_train_ds, 
            epochs=100, 
            validation_data=tib_eng_val_ds,
            callbacks=[loss_callback]
            )

            # save the model
            tib_eng_translator.save('/home/j/Documents/Projects/Iron-Bridge/lotsawa/models/tib-eng-translator-0.4.1.'+str(i)+'.keras')


In [15]:
train(tib_eng_translator)

TRAINING BATCH 0
499972 total pairs
474974 training pairs
24998 validation pairs
BATCH SIZE 64
Epoch 1/100
7422/7422 [==============================] - 329s 43ms/step - loss: 0.9229 - accuracy: 0.8782 - val_loss: 0.7436 - val_accuracy: 0.9106
Epoch 2/100
7422/7422 [==============================] - 305s 41ms/step - loss: 0.8834 - accuracy: 0.8854 - val_loss: 0.7299 - val_accuracy: 0.9145
Epoch 3/100
7422/7422 [==============================] - 304s 41ms/step - loss: 0.8518 - accuracy: 0.8898 - val_loss: 0.7200 - val_accuracy: 0.9165
Epoch 4/100
7422/7422 [==============================] - 304s 41ms/step - loss: 0.8309 - accuracy: 0.8928 - val_loss: 0.7009 - val_accuracy: 0.9181
Epoch 5/100
7422/7422 [==============================] - 303s 41ms/step - loss: 0.8199 - accuracy: 0.8942 - val_loss: 0.6867 - val_accuracy: 0.9210
Epoch 6/100
7422/7422 [==============================] - 301s 41ms/step - loss: 0.8054 - accuracy: 0.8961 - val_loss: 0.6796 - val_accuracy: 0.9212
Epoch 7/100
7422/

2023-10-29 18:37:02.190506: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 329.59MiB (rounded to 345600000)requested by op tib_eng_translator/model_1/dense/Softmax
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-10-29 18:37:02.190587: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-10-29 18:37:02.190626: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 108, Chunks in use: 105. 27.0KiB allocated for chunks. 26.2KiB in use in bin. 443B client-requested in use in bin.
2023-10-29 18:37:02.190657: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-10-29 18:37:02.190688: I tensorf

ResourceExhaustedError: Graph execution error:

Detected at node tib_eng_translator/model_1/dense/Softmax defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/j/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/j/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1053, in launch_instance

  File "/home/j/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/home/j/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/j/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 516